In [1]:
from inference import ScratchModel
from IPython.display import clear_output

model = ScratchModel()

In [3]:
question = "How can I apply to SUNY Brockport?"

generator = model(question, stream=True, max_tokens=10)

for token in generator:
    print(token)
    clear_output(wait=True)

Tf.Tensor(b' you can find the suny brockport, you can reach', shape=(), dtype=string)
